# SPH Sampling

In [1]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere
from metrics import *
from average_meter import AverageMeter

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [13]:
#dataset_path = '/mnt/data/datasets/nuscenes/v1.0-mini/'
dataset_path = '/media/berlukas/T7 Touch/data/nuscenes'
nusc = NuScenes(version='v1.0-trainval', dataroot=dataset_path, verbose=True)
all_cam_strings = ['CAM_FRONT','CAM_FRONT_RIGHT','CAM_BACK_RIGHT','CAM_BACK','CAM_BACK_LEFT','CAM_FRONT_LEFT']
#all_cam_strings = ['CAM_FRONT_LEFT','CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']
lidar_string = 'LIDAR_TOP'

Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
Done loading in 32.750 seconds.
Reverse indexing ...
Done reverse indexing in 6.7 seconds.


In [2]:
def CreateGrid(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([2, n_grid, n_grid])
    for i in range(n_grid):
        for j in range(n_grid):
            points[0, i, j] = (np.pi*(2*i+1))/(4*bw)
            points[1, i, j] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points

def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points

def ConvertGridToEuclidean(grid):
    cart_grid = np.zeros([3, grid.shape[1], grid.shape[2]])
    cart_grid[0,:,:] = np.multiply(np.sin(grid[0, :,:]), np.cos(grid[1,:,:]))
    cart_grid[1,:,:] = np.multiply(np.sin(grid[0, :, :]), np.sin(grid[1, :, :]))
    cart_grid[2,:,:] = np.cos(grid[0, :, :])    
    return cart_grid

def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def rgb_to_greyscale(r,g,b):
    return 0.2126*r + 0.7152*g + 0.0722*b

def transform_from_pcl_to_cam(nusc, pointsensor, cam, pc):
    # Points live in the point sensor frame. So they need to be transformed via global to the image plane.
    # First step: transform the pointcloud to the ego vehicle frame for the timestamp of the sweep.
    cs_record = nusc.get('calibrated_sensor', pointsensor['calibrated_sensor_token'])
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record['translation']))

    # Second step: transform from ego to the global frame.
    poserecord = nusc.get('ego_pose', pointsensor['ego_pose_token'])
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
    pc.translate(np.array(poserecord['translation']))

    # Third step: transform from global into the ego vehicle frame for the timestamp of the image.
    poserecord = nusc.get('ego_pose', cam['ego_pose_token'])
    pc.translate(-np.array(poserecord['translation']))
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix.T)

    # Fourth step: transform from ego into the camera.
    cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
    pc.translate(-np.array(cs_record['translation']))
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)
    
    return pc

def transform_from_cam_to_pcl(nusc, pointsensor, cam, pc):
    # Transform from the camera into the vehicle's ego frame
    cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])    
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record['translation']))
    
    # Transform from the ego frame (cam) to the global frame
    poserecord = nusc.get('ego_pose', cam['ego_pose_token'])
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
    pc.translate(np.array(poserecord['translation']))
        
    # Transform from the global frame to the ego frame of the LiDAR.
    poserecord = nusc.get('ego_pose', pointsensor['ego_pose_token'])    
    pc.translate(-np.array(poserecord['translation']))
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix.T)
        
    # Transform from the ego frame (LiDAR) to the LiDAR frame
    cs_record = nusc.get('calibrated_sensor', pointsensor['calibrated_sensor_token'])    
    pc.translate(-np.array(cs_record['translation']))
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)
    
    return pc

def project_pc_on_cam(pc, cam_intrinsics, depths):
    # Take the actual picture (matrix multiplication with camera-matrix + renormalization).
    points = view_points(pc.points[:3, :], cam_intrinsics, normalize=True)

    # Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
    # Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
    # casing for non-keyframes which are slightly out of sync.
    min_dist = 0.0001
    mask = np.ones(depths.shape[0], dtype=bool)
    mask = np.logical_and(mask, depths < -1.0)
    mask = np.logical_and(mask, points[0, :] > 1)
    mask = np.logical_and(mask, points[0, :] < im.size[0] - 1)
    mask = np.logical_and(mask, points[1, :] > 1)
    mask = np.logical_and(mask, points[1, :] < im.size[1] - 1)
    return points, mask

def sample_mono_from_image(im, pc, mask):
    n_mask = len(mask)
    #print(f"mask len is {n_mask}")
    for i in range(0,n_mask):    
        visible = mask[i]
        if not visible:
            continue
        cur_point = (points[0,i], points[1,i])    
        px = im.getpixel(cur_point)    
        intensity = rgb_to_greyscale(px[0], px[1], px[2])
        pc.points[3,i] = intensity
    return pc

def visualizeRawPointCloud(cloud, jupyter = False):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])

    if jupyter:
        self.__visualizeJupyter(pcd)
    else:
        o3d.visualization.draw_geometries([pcd])
        
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def writeRawPointCloud(cloud, filename):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.io.write_point_cloud(filename, pcd)

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        """
        Returns the number of dimensions.
        :return: Number of dimensions.
        """
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)

In [ ]:
print(f"We have {len(nusc.scene)} scenes in the dataset.")
#for scene in nusc.scene:
scene = nusc.scene[1]
first_sample_token = scene['first_sample_token']
sample = nusc.get('sample', first_sample_token)
pointsensor = nusc.get('sample_data', sample['data'][lidar_string])
bw = 100
scale = 100
pc = SamplingPointCloud.from_bw(bw, scale)

for cam_str in all_cam_strings:  
    print(f"Sampling for cam {cam_str}")
    cam = nusc.get('sample_data', sample['data'][cam_str])
    im = Image.open(osp.join(dataset_path, cam['filename']))    

    # Transform pointcloud into the camera frame
    pc = transform_from_pcl_to_cam(nusc, pointsensor, cam, pc)
    # Grab the depths (camera frame z axis points away from the camera).
    depths = pc.points[2, :]
#    print(f"depths {depths[1:20]}")

    # Project the points onto the image plane
    cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
    intrinsics = np.array(cs_record['camera_intrinsic'])
    points, mask = project_pc_on_cam(pc, intrinsics, depths)
    filtered_points = points[:, mask]

    # Sample the intensity values from the image
    pc = sample_mono_from_image(im, pc, mask)    
    
    # Transform back into the LiDAR frame
    pc = transform_from_cam_to_pcl(nusc, pointsensor, cam, pc)
    #visualizeRawPointCloud(pc.points.T)
        
visualizeRawPointCloud(pc.points.T / 100)
#writeRawPointCloud(pc.points.T, 'test_full.ply')
#writeRawPointCloud(pc.points.T/100, 'test_small.ply')

In [ ]:
# Init axes.
dot_size = 2
fig, ax = plt.subplots(1, 1, figsize=(9, 16))
ax.imshow(im)
ax.scatter(filtered_points[0, :], filtered_points[1, :], s=dot_size)
ax.axis('off')

--------------------
# Convert dataset to PLY

In [3]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
# export_ds = '/media/scratch/berlukas/nuscenes'
# export_images = export_ds + '/SPH_IMAGES/'
# export_clouds = export_ds + '/SPH_CLOUDS/'
bw = 100
scale = 100

In [15]:
n_scenes = len(nusc.scene)
per_chunk = int(n_scenes / 3)

chunk1_start = 0
chunk1_end = per_chunk
chunk2_start = chunk1_end
chunk2_end = chunk2_start + per_chunk
chunk3_start = chunk2_end
chunk3_end = n_scenes

print(f"n_scenes = {n_scenes}")
print(f"chunk 1 [{chunk1_start}, {chunk1_end}]")
print(f"chunk 2 [{chunk2_start}, {chunk2_end}]")
print(f"chunk 3 [{chunk3_start}, {chunk3_end}]")

n_scenes = 850
chunk 1 [0, 283]
chunk 2 [283, 566]
chunk 3 [566, 850]


## Convert the images to PLY files

In [ ]:
def progresser_images(sample, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
    sample_sphere = ImageSphere(sample)
    return sample_sphere.sampleUsingGrid(grid)

In [ ]:
n_scenes = len(nusc.scene)
print(f"Processing {len(nusc.scene)} scenes in the dataset.")

all_sph_images = [None] * n_scenes
for i in tqdm(range(0, n_scenes)):
    scene = nusc.scene[i]
    first_sample_token = scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)
    pointsensor = nusc.get('sample_data', sample['data'][lidar_string])
    pc = SamplingPointCloud.from_bw(bw, scale)

    for cam_str in all_cam_strings:  
        #print(f"Sampling for cam {cam_str}")
        cam = nusc.get('sample_data', sample['data'][cam_str])
        im = Image.open(osp.join(dataset_path, cam['filename']))    

        # Transform pointcloud into the camera frame
        pc = transform_from_pcl_to_cam(nusc, pointsensor, cam, pc)
        # Grab the depths (camera frame z axis points away from the camera).
        depths = pc.points[2, :]

        # Project the points onto the image plane
        cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
        intrinsics = np.array(cs_record['camera_intrinsic'])
        points, mask = project_pc_on_cam(pc, intrinsics, depths)
        filtered_points = points[:, mask]

        # Sample the intensity values from the image
        pc = sample_mono_from_image(im, pc, mask)    

        # Transform back into the LiDAR frame
        pc = transform_from_cam_to_pcl(nusc, pointsensor, cam, pc)
        #visualizeRawPointCloud(pc.points.T)
        
        #filename = f"{export_images}images{i}.ply"
        #writeRawPointCloud(pc.points.T, filename)
    all_sph_images[i] = pc.points.T / scale
    
print(f"Loading complete. Computing features...")    
grid, _ = DHGrid.CreateGrid(bw)
img_features = process_map(partial(progresser_images, grid=grid), all_sph_images, max_workers=8)

filename = f"{export_ds}/images.npy"
np.save(filename, img_features)
print(f"Wrote features to {filename}")

## Convert the pointclouds to PLYs

In [16]:
def progresser(sample, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
    sample_sphere = Sphere(sample)
    return sample_sphere.sampleUsingGrid(grid)

def writeRawPointCloud(cloud, filename):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.io.write_point_cloud(filename, pcd)
    
def get_point_cloud_from_token(sample_token):
    sample = nusc.get('sample', sample_token)
    pointsensor = nusc.get('sample_data', sample['data'][lidar_string])
    # Load a single lidar point cloud.
    pcl_path = osp.join(dataset_path, pointsensor['filename'])
    pc = LidarPointCloud.from_file(pcl_path)
    return pc.points.T, sample['next']

In [ ]:
start = chunk3_start
end = chunk3_end

print(f"Processing scenes from {start} to {end}.")

all_point_clouds = []
for i in tqdm(range(start, end)):
    scene = nusc.scene[i]
    first_sample_token = scene['first_sample_token']    
    pc, next_sample_token = get_point_cloud_from_token(first_sample_token)
    all_point_clouds.append(pc)
    
    while next_sample_token != '':    
        pc, next_sample_token = get_point_cloud_from_token(next_sample_token)    
        all_point_clouds.append(pc)        
    
print(f"Loaded {len(all_point_clouds)} pointclouds. Computing features...")    
#visualizeRawPointCloud(all_point_clouds[0])
grid, _ = DHGrid.CreateGrid(bw)
pcl_features = process_map(partial(progresser, grid=grid), all_point_clouds, max_workers=8, chunksize=100)

filename = f"{export_ds}/clouds3.npy"
np.save(filename, pcl_features)
print(f"Wrote features to {filename}")

#for i in range(0, n_scenes):   
#    filename = f"{export_clouds}cloud{i}.ply"
#    writeRawPointCloud(pcl_features[i], filename)

## LiDAR segmentation

In [17]:
def get_semantic_point_cloud_from_token(sample_token):
    sample = nusc.get('sample', sample_token)
    pointsensor = nusc.get('sample_data', sample['data'][lidar_string])
    # Load a single lidar point cloud.
    pcl_path = osp.join(dataset_path, pointsensor['filename'])
    pc = LidarPointCloud.from_file(pcl_path)
    points_xyz = pc.points.T[:,0:3]
    
    # Load the semantic segmentation.
    sample_data_token = sample['data'][lidar_string]
    lidarseg_labels_filename = osp.join(nusc.dataroot, nusc.get('lidarseg', sample_data_token)['filename'])
    points_label = np.fromfile(lidarseg_labels_filename, dtype=np.uint8)  # [num_points]

    # Combine the two.
    return np.column_stack((points_xyz, points_label)), sample['next']

In [19]:
start = chunk3_start
end = chunk3_end

print(f"Processing scenes from {start} to {end}.")

all_sem_clouds = []
for i in tqdm(range(start, end)):
    scene = nusc.scene[i]
    first_sample_token = scene['first_sample_token']
    pc, next_sample_token = get_semantic_point_cloud_from_token(first_sample_token)
    all_sem_clouds.append(pc)
    
    while next_sample_token != '':    
        pc, next_sample_token = get_semantic_point_cloud_from_token(next_sample_token)    
        all_sem_clouds.append(pc)   
    
print(f"Loading complete. Computing features...")    
grid, _ = DHGrid.CreateGrid(bw)
sem_features = process_map(partial(progresser, grid=grid), all_sem_clouds, max_workers=8, chunksize=100)    

filename = f"{export_ds}/sem_clouds3.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}")

Processing scenes from 566 to 850.



Loading complete. Computing features...



Wrote features to /mnt/data/datasets/nuscenes/processed/sem_clouds3.npy


In [11]:
# n_clouds = len(pcl_features)
# n_images = len(img_features)
# assert n_clouds > 0
# assert n_clouds == n_images

i = 15
# cur_cloud = pcl_features[i]
cur_sem_cloud = sem_features[i]
# cur_image = img_features[i]
# print(f"current spherical cloud shape: {cur_cloud.shape} and current spherical image shape {cur_image.shape}")
# cur_cloud = np.reshape(cur_cloud, (2, -1)).T
cur_sem_cloud = np.reshape(cur_sem_cloud, (2, -1)).T
# cur_image = np.reshape(cur_image, (1, -1)).T
# print(f"current reshaped cloud shape {cur_cloud.shape} and current reshaped image shape {cur_image.shape}")

pc = SamplingPointCloud.from_bw(bw, 1)
points_xyz = pc.points.T[:,0:3]
print(f"sampling pointcloud shape is {points_xyz.shape}")
# points_xyzi = np.column_stack((points_xyz, cur_cloud[:,1]))
# points_xyzp = np.column_stack((points_xyz, cur_image))
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,1]))

visualizeRawPointCloud(points_xyzl)
#visualizeRawPointCloud(points_xyzp)
#visualizeRawPointCloud(points_xyzl)

#print(f"reshaped cloud shape is {cloud.shape}")
#visualizeRawPointCloud(cloud)

sampling pointcloud shape is (40000, 3)


---------------------
## Check decoded signals

In [4]:
def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd    

def create_cloud_pc(cloud, trans = 0):
    cloud[:,0] = cloud[:,0] + trans
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd

def compare_estimation_sphere(decoded, gt, bw = 100):  
    decoded_pc = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pc = create_spherical_pc(gt, trans=2.5, bw=bw)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])    
    
def compare_estimation_clouds(decoded, gt, bw = 100):  
    decoded_pc = create_cloud_pc(decoded, trans=0)
    gt_pc = create_cloud_pc(gt, trans=100)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])
    
def backproject_cloud(spherical, distance, bw = 100):    
    grid, _ = DHGrid.CreateGrid(bw)
    n_points = grid.shape[1] * grid.shape[2]
    cart_sphere = np.zeros([n_points, 4])
    k = 0
    for i in range(0, grid.shape[1]):
        for j in range(0, grid.shape[2]):
            dist = distance[i,j]
            if dist <= 0:
                continue
            cart_sphere[k,0] = dist * np.multiply(np.cos(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,1] = dist * np.multiply(np.sin(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,2] = dist * np.cos(grid[0,i,j])
            cart_sphere[k,3] = spherical[i,j]
            k = k + 1
    return cart_sphere  

In [5]:
decoded_filename = f"{export_ds}/decoded.npy"
decoded_gt_filename = f"{export_ds}/decoded_gt.npy"
decoded_input_filename = f"{export_ds}/decoded_input.npy"

decoded_features = np.load(decoded_filename)
decoded_gt = np.load(decoded_gt_filename)
decoded_input = np.load(decoded_input_filename)
print(f"shape of the decoded/gt/input signal: {decoded_features.shape}/{decoded_gt.shape}/{decoded_input.shape}.")

shape of the decoded/gt/input signal: (1708, 9, 200, 200)/(1708, 200, 200)/(1708, 2, 200, 200).


In [6]:
n_decoded = decoded_features.shape[0]

avg_pixel_acc = AverageMeter()
avg_pixel_acc_per_class = AverageMeter()
avg_jacc = AverageMeter()
avg_dice = AverageMeter()
for i in range(5, 10):    
    cur_decoded = decoded_features[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_sem_cloud = decoded_gt[i, :, :]
    cur_input = decoded_input[i, :, :]
    
    pred_segmentation = torch.from_numpy(cur_decoded).cuda().int()
    gt_segmentation = torch.from_numpy(cur_sem_cloud).cuda().int()
    pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(gt_segmentation, pred_segmentation, num_classes = 9)
    avg_pixel_acc.update(pixel_acc)
    avg_pixel_acc_per_class.update(pixel_acc_per_class)
    avg_jacc.update(jacc)
    avg_dice.update(dice)
        
    cur_decoded = np.reshape(cur_decoded, (1, -1)).T    
    cur_sem_cloud = np.reshape(cur_sem_cloud, (1, -1)).T    
    
    compare_estimation_sphere(cur_decoded, cur_sem_cloud, 100)

print(f'overall pixel acc = {avg_pixel_acc.avg}')
print(f'pixel acc per class = {avg_pixel_acc_per_class.avg}')
print(f'mean jaccard index = {avg_jacc.avg}')
print(f'mean dice coeff = {avg_dice.avg}')

RuntimeError: cuda runtime error (999) : unknown error at /pytorch/aten/src/THC/THCGeneral.cpp:47

In [13]:
bw = 100
for i in range(350, 351):    
    cur_decoded = decoded_features[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_sem_sphere = decoded_gt[i, :, :]    
    
    cur_decoded = np.reshape(cur_decoded, (1, -1)).T    
    cur_sem_sphere = np.reshape(cur_sem_sphere, (1, -1)).T    
    
    compare_estimation_sphere(cur_decoded, cur_sem_sphere, 100)

## Visualize Semantic Pointcloud

In [15]:
bw = 100
for i in range(350, 360):    
    cur_decoded = decoded_features[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_sem_sphere = decoded_gt[i, :, :]
    cur_input = decoded_input[i, :, :]
    
    est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
    gt_cloud = backproject_cloud(cur_sem_sphere, cur_input[0,:,:], bw)
    compare_estimation_clouds(est_cloud, gt_cloud)

In [18]:
np.count_nonzero(decoded_gt[0, :, :] == decoded_gt[3,:,:])

32826